# Assignment 1: Transformations and Representations

Roll number: 2019101068


# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. 
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Make sure any extra files you that you need to submit, place it in *'results'* folder.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on **23/08/2022 at 11:55pm**. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``<roll_number>_MR2022_<assignment_number>.zip``

## 1. 3D Data and Open3D

1. Please find mesh files in **data/Q1** folder. Using these mesh files and your own creativity/visualisation, create a "Table" **pointcloud** scene. The table scene should be realistic, scaled appropiately. Use all the meshes given in the folder and treat them as objects kept on the table. 

    You are expected to perform different functions on the individual mesh files: first convert the mesh files to pointclouds and on each pointcloud perform operations such as scaling, rotation, translation. Next, visualize them together. The visualization should represent a pointcloud of a realistic table scene. Save the scene as **.pcd** file. 

    **Please do not copy as we may use your contribution to create a table top dataset.**

    Refer below image for example of a table-top scene:

    <img src="img/1.jpeg"  width="500" >
<br>
<br>

2. Use the final table scene pointcloud obtained from part 1. 
    - Use Open3D to generate partial pointclouds from different camera views (at least 4 views). This means, that you need to crop or capture the points in the pointcloud that are visible only from a given viewpoint. 
    - Using these partial pointclouds, you are now expected to generate the full scene pointcloud back by registering the pointclouds to a global frame. Save the partial and reconstructed pointclouds in different files. 
    - **[ BONUS ]** Finally, compute the error using "Chamfer's Distance (CD)" between the ground truth scene pointcloud and the reconstructed pointcloud. Perform an analysis: 
      1. Why is the CD not 0?
      2. How does the CD change as the number of viewpoints increase / decrease? 
      3. Can we optimize the viewpoints (by hit and trial) such that the CD reduces?

Refer the following link for solving Q1:
- Hidden-Point-Removal Open3D API: http://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html#Hidden-point-removal

- Chamfer's Distance:  https://pytorch3d.readthedocs.io/en/latest/modules/loss.html



### Importing important libraries (Some of the common code I have written is in helper.py)

In [26]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import sys
import numpy as np
from copy import deepcopy
import helper
import seaborn as sns

In [27]:
import importlib
importlib.reload(helper)

<module 'helper' from '/home/anmolagarwal/Desktop/mr_assign_1/helper.py'>

### Loading a sample mesh

In [28]:
def generate_table_scene(path_to_save):
    
    # scaling factor for the different objects
    object_names={"table":1, "boat":30,"laptop":40,"trashcan":20,"plane":50,"car":20 }

    NUM_COLORS=8
    COLORS_ARR=sns.color_palette('husl', n_colors=NUM_COLORS)
    bb=[]

    pcd_dict=dict()

    for idx, curr_obj in enumerate(object_names):
        print("curr obj is ", curr_obj)
        PCL_PATH=os.path.join("./data/Q1", f"{curr_obj}.obj")
        mesh = helper.load_mesh(PCL_PATH)
        print(f'Center of mesh is: {mesh.get_center()}')
        print("bounds are : ", mesh.get_min_bound(), mesh.get_max_bound())

        #print(mesh)
        curr_pcl=helper.fetch_pcd_from_mesh(mesh)
        pcd_dict[curr_obj] = curr_pcl

        displacement=pcd_dict[curr_obj].get_center()

        if curr_obj=="table":
            displacement[2]=0


        pcd_dict[curr_obj]=deepcopy(pcd_dict[curr_obj]).translate(tuple([-x for x in displacement]))
        pcd_dict[curr_obj].scale(object_names[curr_obj], center=pcd_dict[curr_obj].get_center())

        pcd_dict[curr_obj].paint_uniform_color(COLORS_ARR[idx])


        print("############")


    '''TABLE'''
    curr_pcd=pcd_dict['table']
    curr_pcd.rotate(curr_pcd.get_rotation_matrix_from_axis_angle([0, 0,-np.pi/2]), center=(0,0,0))

    curr_pcd.points=o3d.utility.Vector3dVector(np.asarray(curr_pcd.points) * np.array([2.4, 2, 1.]) )

    height_of_table = curr_pcd.get_max_bound()[2]






    '''BOAT'''
    curr_pcd=pcd_dict['boat']
    curr_pcd.rotate(curr_pcd.get_rotation_matrix_from_axis_angle([np.pi/2, 0,0]), center=(0,0,0))

    # make boat on table
    x_min, y_min, z_min = curr_pcd.get_min_bound()
    diff= height_of_table  - z_min
    curr_pcd.translate((-40, 0, diff))




    '''PLANE'''
    curr_pcd=pcd_dict['plane']
    curr_pcd.rotate(curr_pcd.get_rotation_matrix_from_axis_angle([np.pi/2, 0,0]), center=(0,0,0))
    x_min, y_min, z_min = curr_pcd.get_min_bound()
    diff= height_of_table  - z_min
    curr_pcd.translate((40, -20, diff))



    '''CAR'''
    curr_pcd=pcd_dict['car']
    curr_pcd.rotate(curr_pcd.get_rotation_matrix_from_axis_angle([np.pi/2, 0,0]), center=(0,0,0))
    x_min, y_min, z_min = curr_pcd.get_min_bound()
    diff= height_of_table  - z_min
    curr_pcd.translate((-30, 10, diff))



    '''Laptop'''
    curr_pcd=pcd_dict['laptop']

    curr_pcd.rotate(curr_pcd.get_rotation_matrix_from_axis_angle([0, 0,np.pi]), center=(0,0,0))
    curr_pcd.rotate(curr_pcd.get_rotation_matrix_from_axis_angle([-np.pi/2, 0,0]), center=(0,0,0))

    x_min, y_min, z_min = curr_pcd.get_min_bound()
    diff= height_of_table  - z_min

    curr_pcd.translate((0, 0, diff))



    '''Trashcan'''
    curr_pcd=pcd_dict['trashcan']
    curr_pcd.rotate(curr_pcd.get_rotation_matrix_from_axis_angle([np.pi/2, 0,0]), center=(0,0,0))
    x_min, y_min, z_min = curr_pcd.get_min_bound()
    diff= height_of_table  - z_min
    curr_pcd.translate((40, 20, diff))






    #######################################################

    def populate_bounding_boxes():
        for idx, curr_obj in enumerate(object_names):


            axis_aligned_bounding_box = pcd_dict[curr_obj].get_axis_aligned_bounding_box()
            axis_aligned_bounding_box.color = (1, 0, 0)
            bb.append(axis_aligned_bounding_box)
            print("############")

    #populate_bounding_boxes()
    
    for curr_obj in pcd_dict:
        print("Key is ", curr_obj)
        print("Max is ", pcd_dict[curr_obj].get_max_bound())
        print("Min is ", pcd_dict[curr_obj].get_min_bound())
        print("####")
    #o3d.visualization.draw_geometries([{"name": x, "geometry":pcd_dict[x]} for x in pcd_dict]+bb, show_ui=True)


    master_pcd=o3d.geometry.PointCloud()
    master_pcd.points = o3d.utility.Vector3dVector(
                                            np.concatenate(
                                                [np.asarray(x.points) for x in pcd_dict.values()]
                                            )
                                                )           


    o3d.visualization.draw_geometries([master_pcd])
    o3d.io.write_point_cloud(path_to_save, master_pcd)

#### Uncomment below line to generate point cloud

In [29]:
#generate_table_scene("rand.pcd")

curr obj is  table
Center of mesh is: [89.0884239  90.86248469 12.04770164]
bounds are :  [75.2928009  71.17710114  0.        ] [102.85199738 110.54699707  29.1406002 ]
############
curr obj is  boat
Center of mesh is: [ 0.00070581 -0.00132877  0.04475134]
bounds are :  [-0.122948 -0.096302 -0.699781] [0.118397   0.12895501 0.24415401]
############
curr obj is  laptop
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D IN

### Load the generated point cloud

In [30]:
path="./results/Q1/table_scene.pcd"
pcd = o3d.io.read_point_cloud(path)

In [31]:
pcd

PointCloud with 239308 points.

### Axis colors: Red = X. Green = Y. Blue = Z.

### Key is  table

Max is  [47.33165627 27.59973735 29.1406002 ]

Min is  [-47.15609398 -27.5186556    0.        ]

####
### Key is  boat

Max is  [-36.39824266  15.54964489  35.81709716]

Min is  [-43.63859253 -12.63645464  29.1406002 ]

####
### Key is  laptop

Max is  [15.08221028  6.31626625 46.51891971]

Min is  [-15.16402962 -13.24956646  29.1406002 ]

####
### Key is  trashcan

Max is  [45.41938541 25.50159296 41.69364655]

Min is  [34.44425012 14.54439955 29.1406002 ]

#####
## Key is  plane

Max is  [ 48.49011257 -12.15910415  34.97930526]

Min is  [ 31.49486214 -28.34294406  29.1406002 ]

####
### Key is  car

Max is  [-26.29101459  19.06143285  34.39037717]

Min is  [-33.7436198    1.25992071  29.1406002 ]

####

## Different viewpoints

In [44]:
labels=["from_bottom", "from_top", "from_dustbin_edge", "from_behind_laptop", "between_boat_and_car"]

In [35]:
camera_coordinates_to_try=[
    [0,0, -10], # view bottom half of the table only
    [0, 0, 100], # view top half of the table only
    [48.48938751, 27.63839149, 46.5189209 ], # from dustbin side
    [0, 60, 30 ], # from behind laptop , 
    [-35,  10, 30 ], # from the small space between the boat and the car
    
]

In [45]:
def get_pcds(camera_coordinates, pcd):
    
    radius = diameter * 100
    a_pcd=deepcopy(pcd)

    _, pt_map = pcd.hidden_point_removal(camera_coordinates, radius)

    print("Visualize result")
    a_pcd = pcd.select_by_index(pt_map)
    #o3d.visualization.draw_geometries([a_pcd])
    return a_pcd

In [49]:
def make_pcds():
    for scene_name, curr_coords in zip(labels, camera_coordinates_to_try):
        print("SCENE IS ", scene_name)
        curr_pcd=get_pcds(curr_coords, pcd)
        o3d.io.write_point_cloud(os.path.join("./", f"{scene_name}.pcd"), curr_pcd)
        print("#######")
        #break
    

#### Uncomment to generate

In [51]:
#make_pcds()

SCENE IS  from_bottom
Visualize result
#######
SCENE IS  from_top
Visualize result
#######
SCENE IS  from_dustbin_edge
Visualize result
#######
SCENE IS  from_behind_laptop
Visualize result
#######
SCENE IS  between_boat_and_car
Visualize result
#######


### Concatenate and see

In [60]:
ROOT_PATH="./results/Q1/pcds"
all_paths=os.listdir(ROOT_PATH)

In [58]:
all_paths

['from_bottom.pcd',
 'from_top.pcd',
 'from_dustbin_edge.pcd',
 'between_boat_and_car.pcd',
 'from_behind_laptop.pcd']

In [59]:
pcds=[]

In [67]:
all_points_set=set()

In [77]:
tot_points=0
tot_unique_points=0
for curr_pcd_path in all_paths:
    print(curr_pcd_path)
    pcd_now = o3d.io.read_point_cloud(os.path.join(ROOT_PATH, curr_pcd_path))
    curr_points=np.array(pcd_now.points)
    
    for curr_point in curr_points:
        all_points_set.add(tuple(curr_point.tolist()))
    print("Tot pts is ", len(curr_points))
    tot_points+=len(curr_points)
    print("#######")

from_bottom.pcd
Tot pts is  14399
#######
from_top.pcd
Tot pts is  36503
#######
from_dustbin_edge.pcd
Tot pts is  29317
#######
between_boat_and_car.pcd
Tot pts is  34478
#######
from_behind_laptop.pcd
Tot pts is  15909
#######


In [78]:
tot_points

130606

In [81]:
print("Total points gathered: ", tot_points)
print("Total UNIQUE points gathered: ", len(all_points_set))
print("Total points EXISTING: ", len(np.asarray(pcd.points)))

Total points gathered:  130606
Total UNIQUE points gathered:  91465
Total points EXISTING:  239308


### Reconstructing all PCDs

In [82]:
       

path="./results/Q1/pcds/between_boat_and_car.pcd"
new_pcd = o3d.geometry.PointCloud()
new_pcd.points = o3d.utility.Vector3dVector(np.array(list(all_points_set)))


In [83]:
o3d.io.write_point_cloud("./results/Q1/reconstructed_pcd.pcd", new_pcd)

True